In [ ]:
import os
from pathlib import Path
import func_proc_filepath as mFILE
import func_proc_image as mIMAGE
import cv2
from statistics import mean

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def main():
    # root folder
    dir_root = str(Path(Path().resolve()).parent)

    # input: folder path including original images
    dir_img = dir_root + "\\dataset\\expw_images\\cropped"

    # output: folder path
    dir_out = dir_root + "\\dataset\\expw_images\\corrected"
    if os.path.isdir(dir_out) == False:
        os.makedirs(dir_out)
    
    files_list = mFILE.search_files(dir_img, valid_names=[".jpg"], invalid_names=["wuert"], ext=None, recursive=False)
    log = "number of files: {0}".format(len(files_list))
    print(log)
    
    # set hsv parameter
    h_value = 14
    s_value = 105
    v_value = 165
    
    max_cnt = len(files_list)
    
    for i in range(len(files_list)):
    #for i in range(200):
        fp = files_list[i]
        name = os.path.splitext(os.path.basename(fp))[0]
        
        img_src = cv2.imread(fp)
        
        #img = cv2.cvtColor(img_src, cv2.COLOR_BGR2RGB)
        #display_cv_image(img_src, format='.png')
        
        cnter_img = img_src[60:240, 60:240]        
        hsv = cv2.cvtColor(cnter_img, cv2.COLOR_BGR2HSV_FULL)
        h_mean_tmp = hsv[:,:,0].mean()
        s_mean_tmp = hsv[:,:,1].mean()
        v_mean_tmp = hsv[:,:,2].mean()
        
        img = mIMAGE.crop_image(img_src)
        hsv2 = cv2.cvtColor(img, cv2.COLOR_BGR2HSV_FULL)
        
        # hsv
        hsv2[:,:,0] = h_value
        hsv2 = mIMAGE.changedS(hsv2, 1, round(s_value - s_mean_tmp))
        hsv2 = mIMAGE.changedV(hsv2, 1, round(v_value - v_mean_tmp))
        # contrast
        # histogram
        hist = cv2.calcHist([hsv2],[2],None,[256],[0,256]) 
        clahe = cv2.createCLAHE(clipLimit=1,tileGridSize=(4,4))
        hsv2[:,:,2] = clahe.apply(hsv2[:,:,2]) 
        
        img_out = cv2.cvtColor(hsv2,cv2.COLOR_HSV2BGR_FULL) # 色空間をHSVからBGRに変換
        img_out = mIMAGE.fill_image(img_out, 300, 300)
        
        if i <= 20:
            mIMAGE.display_cv_image(img_src, format='.png')
            mIMAGE.display_cv_image(img_out, format='.png')
        
        file_out = dir_out + "\\" + name + ".jpg"
        cv2.imwrite(file_out, img_out)
        
        log = "{0}/{1}, {2} is finished".format(i, max_cnt-1, name)
        print(log)
    

In [ ]:
if __name__ == "__main__":
    main()